# Studying the effect of biomass compositions on leaf metabolism

- This is a simple implementation of the [Yuan et. al., 2016](https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2016.00537/full) study for C3 leaf system
- Yuan et. al. collected biomass composition of heterotrophic Arabidopsis cells from tissue cultures reported by [Poolman et al.](https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2016.00537/full#B50), [de Oliveira Dal'Molin et al.](https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2016.00537/full#B16) and [Arnold & Nikoloski](https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2016.00537/full#B2)
- The effect of changing biomass compositions in the diel leaf C3 model will be studied by comparing flux distributions

## Step 1: Install cobrapy and import the diel leaf C3 model
- Run `%pip install cobra` to install cobrapy
- Use wget using the syntax `!wget link-to-file` to retrieve the models and functions required for this analysis  


In [1]:
%pip install cobra --quiet

!wget -q https://raw.githubusercontent.com/sshameer/DielLeafModeling/refs/heads/main/Models/C3_model.sbml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 15.3 MB/s eta 0:00:00


## Step 2: Read the C3 model and constrain it for 500 PPFD light
- A cobra model can be generated from an SBML file using the `read_sbml_model` function in the cobrapy `io` module
- The syntax is `model_name = read_sbml_model(file_name)`
- see [another IPython notebook](https://github.com/sshameer/DielLeafModeling/blob/main/Case%20studies/dielC3_script.ipynb) for details on constraining the model for 500 PPFD light conditions

In [2]:
from cobra.io import read_sbml_model
model = read_sbml_model("C3_model.sbml")


# constraining GPT transport reaction
model.reactions.get_by_id("G6P_Pi_pc1").lower_bound= 0
model.reactions.get_by_id("G6P_Pi_pc1").upper_bound= 0
model.reactions.get_by_id("G6P_Pi_pc2").lower_bound= 0
model.reactions.get_by_id("G6P_Pi_pc2").upper_bound= 0

# constraining starch phosphorylation
model.reactions.get_by_id("RXN_1826_p1").lower_bound= 0
model.reactions.get_by_id("RXN_1826_p1").upper_bound= 0
model.reactions.get_by_id("RXN_1826_p2").lower_bound= 0
model.reactions.get_by_id("RXN_1826_p2").upper_bound= 0

# constraining sucrose accumulation to model a starch storing leaf
model.reactions.get_by_id("SUCROSE_v_dielTransfer").bounds = (0,0)

# Setting photon uptake
PPFD = 500
model.reactions.Photon_tx1.upper_bound = PPFD
model.reactions.Photon_tx1.lower_bound = 0
# Setting maintenance cost
VATPase = 0.0049*PPFD+2.7851
model.reactions.get_by_id("ATPase_tx1").bounds = (VATPase,VATPase)


- Next check if the model is able to provide a feasible solution when running pFBA
- pFBA can be performed using the `pfba` function found in cobrapy `flux_analysis` module  
- `model.summary()` can be used to quickly check the results of the simulation

In [3]:
from cobra.flux_analysis import pfba
C3sol = pfba(model)

model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,35.14,1,100.00%
WATER_e1,H2O_tx1,31.55,0,0.00%
NITRATE_e1,Nitrate_tx1,0.2171,0,0.00%
NITRATE_e2,Nitrate_tx2,0.1447,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,2.023,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
SULFATE_e1,SO4_tx1,0.002766,0,0.00%
PROTON_c1,unlProtHYPO_c1,0.3151,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e2,CO2_tx2,-1.846,1,100.00%


## Step 3: Retrieve the three biomass compositions from Yuan et. al., 2016
- Data can be collected in excel files, text files, etc and imported in this notebook to collect biomass composition. Most importantly when collecting data, make sure to identify the metabolite IDs used in this model
- Here python dictionary objects will be made for easy implementation

In [4]:
#Collect AraGEM biomass collected by Yuan et al 2016
AraGEM_dict = {"pALA_b":	0.107,"pARG_b":	0,"pASN_b":	0.39,"pASP_b":	0.162,"pCYS_b":	0.024,"pGLN_b":	0.35,"pGLU_b":	0.39,"pGLY_b":	0.14,"pHIS_b":	0,"pILE_b":	0.06,"pLEU_b":	0.14,"pLYS_b":	0.001,"pMET_b":	0.002,
                "pPHE_b":	0.08,"pPRO_b":	0.01,"pSER_b":	0.21,"pTHR_b":	0.086,"pTRP_b":	0.04,"sTYR_b":	0.03,"pVAL_b":	0.1,"GLC_c":	0.053,"sSUCROSE_b":	0.096,"FRU_c":	0.053,"STARCH_p":	0.265,"PALMITATE_c":	0.026,
                "CELLULOSE_c":	0.305,"Xylan_b":	0.282,"DATP_p":	0.002133,"aDATP_p":	0.000567,"DCTP_p":	0.001501,"aDCTP_p":	0.000399,"DGTP_p":	0.00085,"aDGTP_p":	0.00017,"bDGTP_p":	0.00068,"DTTP_c":	0.001372,
                "aDTTP_c":	0.001428,"AMP_c":	0.0029,"CMP_p":	0.0033,"aGMP_c":	0.000234,"GMP_c":	0.002925,"bGMP_c":	0.000741,"UMP_p":	0.0029}

#Collect AraMeta biomass collected by Yuan et al 2016
AraMeta_dict = {"pALA_b":0.964,"pARG_b":0.118,"pASN_b":0.518,"pASP_b":0.503,"pCYS_b":0.111,"pGLN_b":0.543,"pGLU_b":0.537,"pGLY_b":0.299,"pHIS_b":0.077,"pILE_b":0.188,"pLEU_b":0.372,"pLYS_b":0.08,"pMET_b":0.021,
                "pPHE_b":0.141,"pPRO_b":0.00031,"pSER_b":0.573,"pTHR_b":0.42,"pTRP_b":0.015,"sTYR_b":0.121,"pVAL_b":0.125,"L_ORNITHINE_p":0.00019,"STARCH_p":0.39,"PALMITATE_c":0.879,"CELLULOSE_c":6.838,
                "aDADP_p":0.0084,"DADP_p":0.0316,"DCDP_p":0.03397,"aDCDP_p":0.00903,"aDGDP_p":0.0038,"DGDP_p":0.019,"bDGDP_p":0.0152,"DTMP_c":0.041,"AMP_c":0.038,"CMP_p":0.041,"aGMP_c":0.00216,"GMP_c":0.027,
                "bGMP_c":0.00684,"UMP_p":0.041,}

#Collect AraCore biomass collected by Yuan et al 2016
AraCore_dict = {"pALA_b":0.207,"pARG_b":0.064,"pASN_b":0.062,"pASP_b":0.091,"pCYS_b":0.028,"pGLN_b":0.046,"pGLU_b":0.103,"pGLY_b":0.246,"pHIS_b":0.03,"pILE_b":0.081,"pLEU_b":0.142,"pLYS_b":0.093,"pMET_b":0.03,
                "pPHE_b":0.06,"pPRO_b":0.097,"pSER_b":0.133,"pTHR_b":0.125,"pTRP_b":0.015,"sTYR_b":0.042,"pVAL_b":0.131,"L_ORNITHINE_p":0.015,"MALTOSE_c":0.002,"GLC_c":0.034,"SUCROSE_c":0.045,"FRU_c":0.015,
                "STARCH_p":0.227,"MALONYL_ACP_p":0.78,"ACP_p":-0.78,"CELLULOSE_c":0.364,"DATP_p":0.000553,"aDATP_p":0.000147,"DCTP_p":0.000316,"aDCTP_p":0.000084,"aDGTP_p":0.00004,"DGTP_p":0.0002,"bDGTP_p":0.00016,
                "aDTTP_c":0.000357,"DTTP_c":0.000343,"AMP_c":0.0006,"CMP_p":0.0005,"aGMP_c":0.000024,"GMP_c":0.0003,"bGMP_c":0.000076,"UMP_p":0.0006,"SUC_m":0.001,"FUM_m":0.078,"MAL_m":0.038,"SHIKIMATE_p":0.0003,
                "UREA_m":0.006,"4_AMINO_BUTYRATE_c":0.0002,}

## Step 4: Introduce three new biomass equations based on the collected data

- Remove exisiting biomass equations for simplification
- A reaction can be removed from a model using the `reaction.remove_from_model()` function

In [5]:
for i in [1,2]:
  model.reactions.get_by_id("Biomass_tx"+str(i)).remove_from_model()
  model.reactions.get_by_id("AraCore_Biomass_tx"+str(i)).remove_from_model()

- Introduce a biomass reaction for day and night metabolism, for each biomass composition
- New reactions can be added using the `cobra.core` `Reaction` function
- New metabolites can be added using the `cobra.core` `Metabolite` function

In [6]:
from cobra import Reaction, Metabolite

# Add a AraGEM biomass reaction to each temporal phase
for i in [1,2]:
  rxn = Reaction("AraGEM_Biomass_tx"+str(i))
  for met in AraGEM_dict.keys():
    coeff = AraGEM_dict[met]
    met = model.metabolites.get_by_id(met+str(i))
    rxn.add_metabolites({met:-1*coeff})
  met = Metabolite("X_Biomass_contribution_t"+str(i))
  rxn.add_metabolites({met:1})
  print("Temporal phase "+str(i)+" biomass reaction:")
  print(rxn.reaction)
  model.add_reactions([rxn,])

# Add a AraMeta biomass reaction to each temporal phase
for i in [1,2]:
  rxn = Reaction("AraMeta_Biomass_tx"+str(i))
  for met in AraMeta_dict.keys():
    coeff = AraMeta_dict[met]
    met = model.metabolites.get_by_id(met+str(i))
    rxn.add_metabolites({met:-1*coeff})
  met = Metabolite("X_Biomass_contribution_t"+str(i))
  rxn.add_metabolites({met:1})
  print("Temporal phase "+str(i)+" biomass reaction:")
  print(rxn.reaction)
  model.add_reactions([rxn,])

# Add a AraCore biomass reaction to each temporal phase
for i in [1,2]:
  rxn = Reaction("AraCore_Biomass_tx"+str(i))
  for met in AraCore_dict.keys():
    coeff = AraCore_dict[met]
    met = model.metabolites.get_by_id(met+str(i))
    rxn.add_metabolites({met:-1*coeff})
  met = Metabolite("X_Biomass_contribution_t"+str(i))
  rxn.add_metabolites({met:1})
  print("Temporal phase "+str(i)+" biomass reaction:")
  print(rxn.reaction)
  model.add_reactions([rxn,])


Temporal phase 1 biomass reaction:
0.0029 AMP_c1 + 0.305 CELLULOSE_c1 + 0.0033 CMP_p1 + 0.002133 DATP_p1 + 0.001501 DCTP_p1 + 0.00085 DGTP_p1 + 0.001372 DTTP_c1 + 0.053 FRU_c1 + 0.053 GLC_c1 + 0.002925 GMP_c1 + 0.026 PALMITATE_c1 + 0.265 STARCH_p1 + 0.0029 UMP_p1 + 0.282 Xylan_b1 + 0.000567 aDATP_p1 + 0.000399 aDCTP_p1 + 0.00017 aDGTP_p1 + 0.001428 aDTTP_c1 + 0.000234 aGMP_c1 + 0.00068 bDGTP_p1 + 0.000741 bGMP_c1 + 0.107 pALA_b1 + 0.39 pASN_b1 + 0.162 pASP_b1 + 0.024 pCYS_b1 + 0.35 pGLN_b1 + 0.39 pGLU_b1 + 0.14 pGLY_b1 + 0.06 pILE_b1 + 0.14 pLEU_b1 + 0.001 pLYS_b1 + 0.002 pMET_b1 + 0.08 pPHE_b1 + 0.01 pPRO_b1 + 0.21 pSER_b1 + 0.086 pTHR_b1 + 0.04 pTRP_b1 + 0.1 pVAL_b1 + 0.096 sSUCROSE_b1 + 0.03 sTYR_b1 --> X_Biomass_contribution_t1
Temporal phase 2 biomass reaction:
0.0029 AMP_c2 + 0.305 CELLULOSE_c2 + 0.0033 CMP_p2 + 0.002133 DATP_p2 + 0.001501 DCTP_p2 + 0.00085 DGTP_p2 + 0.001372 DTTP_c2 + 0.053 FRU_c2 + 0.053 GLC_c2 + 0.002925 GMP_c2 + 0.026 PALMITATE_c2 + 0.265 STARCH_p2 + 0.0029 U

- update the `diel_biomass` equation to set the accumulation of biomass in 3:1 as the objective

In [7]:
rxn = model.reactions.get_by_id("diel_biomass")
rxn.add_metabolites({model.metabolites.get_by_id("X_Phloem_contribution_t1"):3,
                     model.metabolites.get_by_id("X_Phloem_contribution_t2"):1,
                     model.metabolites.get_by_id("X_Biomass_contribution_t1"):-3,
                     model.metabolites.get_by_id("X_Biomass_contribution_t2"):-1})

## Step 5: Run pFBA for each biomass composition

- save results of the three simulations in three different solution objects

In [8]:
for i in [1,2]:
  model.reactions.get_by_id("AraGEM_Biomass_tx"+str(i)).bounds = (0,0)
  model.reactions.get_by_id("AraMeta_Biomass_tx"+str(i)).bounds = (0,0)
  model.reactions.get_by_id("AraCore_Biomass_tx"+str(i)).bounds = (0,1000)
  solution1 = pfba(model)

for i in [1,2]:
  model.reactions.get_by_id("AraGEM_Biomass_tx"+str(i)).bounds = (0,1000)
  model.reactions.get_by_id("AraMeta_Biomass_tx"+str(i)).bounds = (0,0)
  model.reactions.get_by_id("AraCore_Biomass_tx"+str(i)).bounds = (0,0)
  solution2 = pfba(model)

for i in [1,2]:
  model.reactions.get_by_id("AraGEM_Biomass_tx"+str(i)).bounds = (0,0)
  model.reactions.get_by_id("AraMeta_Biomass_tx"+str(i)).bounds = (0,1000)
  model.reactions.get_by_id("AraCore_Biomass_tx"+str(i)).bounds = (0,0)
  solution3 = pfba(model)

## Step 6: Comparing the flux distribution by percentage
- Identify key fluxes of leaf metabolism and compare fluxes from the different simulations  
- print results to excel file and observe them

In [9]:
import pandas as pd
reaction_ids = ["CO2_tx1", "CO2_tx2","CO2_pc1","RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1","PHOSPHORIBULOKINASE_RXN_p1","RXN_961_p1","F16ALDOLASE_RXN_p1","GLYCOGENSYN_RXN_p1","FERREDOXIN_NITRITE_REDUCTASE_RXN_p1",
                "ASPAMINOTRANS_RXN_c1","L_ASPARTATE_pc1","GLY3KIN_RXN_p1","GLYCERATE_GLYCOLLATE_pc1","GPH_RXN_p1","GLYCOLLATE_pc1","2KG_MAL_pc1","GAP_Pi_pc1","PEPCARBOX_RXN_c1","Nitrate_ec1","NITRATE_vc1",
                "FERREDOXIN_NITRITE_REDUCTASE_RXN_p1","Glycolate_xc1","RXN_969_x1","GLYCINE_AMINOTRANSFERASE_RXN_x1","SERINE_GLYOXYLATE_AMINOTRANSFERASE_RXN_x1","GCVMULTI_RXN_m1","GLYOHMETRANS_RXN_m1",
                "HYDROXYPYRUVATE_REDUCTASE_RXN_NAD_x1","AraCore_Biomass_tx1","F16BDEPHOS_RXN_c1","Starch_biomass1","RXN_1827_p2","MALTODEG_RXN_c2","GLU6PDEHYDROG_RXN_c2","GAPOXNPHOSPHN_RXN_c2",
                "AraCore_Biomass_tx2","2PGADEHYDRAT_RXN_c2","PEPDEPHOS_RXN_c2","PYRUVDEH_RXN_m2","MALATE_DEH_RXN_m2","CITSYN_RXN_m2","STARCH_p_dielTransfer", "MAL_v_dielTransfer","CIT_v_dielTransfer","NITRATE_v_dielTransfer"]

# to take all common reactions from solution4 (reference)
solution4 = C3sol

flux_comparison = pd.DataFrame({
    "Reaction": reaction_ids,
    "GrowingLeaf_AraCore": [solution1.fluxes.get(rxn, 0.0) for rxn in reaction_ids],
    "GrowingLeaf_AraGEM": [solution2.fluxes.get(rxn, 0.0) for rxn in reaction_ids],
    "GrowingLeaf_AraMeta": [solution3.fluxes.get(rxn, 0.0) for rxn in reaction_ids],
    "MatureLeaf_model": [solution4.fluxes.get(rxn, 0.0) for rxn in reaction_ids]
})



In [10]:
flux_comparison["Reaction"] = flux_comparison["Reaction"].apply(lambda x: x)

# Save the results to an Excel file
flux_comparison.to_excel("flux_comparison_results.xlsx", index=False)


In [11]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
flux_comparison

,Reaction,GrowingLeaf_AraCore,GrowingLeaf_AraGEM,GrowingLeaf_AraMeta,MatureLeaf_model
0,CO2_tx1,31.661,30.193,31.894,35.136
1,CO2_tx2,-2.393,-2.505,-2.867,-1.846
2,CO2_pc1,-35.557,-35.041,-36.353,-42.155
3,RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1,35.520,35.796,36.721,42.205
4,PHOSPHORIBULOKINASE_RXN_p1,47.359,47.729,48.961,56.273
5,RXN_961_p1,11.840,11.932,12.240,14.068
6,F16ALDOLASE_RXN_p1,-17.574,-17.751,-17.992,-20.473
7,GLYCOGENSYN_RXN_p1,1.629,1.667,1.580,1.700
8,FERREDOXIN_NITRITE_REDUCTASE_RXN_p1,4.406,4.901,2.910,0.362
9,ASPAMINOTRANS_RXN_c1,-2.483,-1.432,-2.653,-0.063


As it can be observed the choice of biomass composition only has a small effect on the rate of photosynthesis (CO2_tx1) and other similar key reactions.